### Implementação de uma RNN

In [1]:
#Importando libraries de interesse
import pandas as pd
import numpy as np
import matplotlib #library para gráficos
import matplotlib.pyplot as plt
import sklearn #library de machine learning
from sklearn.datasets import load_breast_cancer #dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time #Infos de tempo

### Função Para Inicialização de Pesos

In [2]:
# Função para inicialização randômica dos parâmetros do modelo
def inicializa_parametros(dims_camada_entrada):
    
    # Dicionário para os parâmetros
    parameters = {}
    
    # Comprimento das dimensões das camadas
    comp = len(dims_camada_entrada)
    
    # Loop pelo comprimento
    for i in range(1, comp):
        
        # Inicialização da matriz de pesos
        parameters["W" + str(i)] = np.random.randn(dims_camada_entrada[i], dims_camada_entrada[i - 1]) * 0.01
        
        # Inicialização do bias
        parameters["b" + str(i)] = np.zeros((dims_camada_entrada[i], 1))
    
    return parameters

In [3]:
# Função sigmóide
'''
Utilizamos a função sigmóide pois permite converter números para valores entre 0 e 1.
Usada para modelos em que temos que prever a probabilidade como uma saída, sendo qualquer item existente no intervalo entre 0 e 1.
'''
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A, Z

In [4]:
# Função de ativação ReLu (Rectified Linear Unit)
'''
Fynção de ativação para utilizarmos na descida do SGD com retropropagação de erros.
'''
def relu(Z):
    A = abs(Z * (Z > 0))
    return A, Z

In [5]:
# Operação de ativação
def linear_activation(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

#"A": matriz com os dados de entrad / "W": matriz de pesos / "b": bias

### Forward Propagation

In [6]:
# Movimento forward
def forward(A_prev, W, b, activation):
    
    # Se a função de ativação for Sigmoid
    if activation == "sigmoid":
        Z, linear_cache = linear_activation(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    # Se for ReLu    
    elif activation == "relu":
        Z, linear_cache = linear_activation(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [7]:
#Combinando Ativação e Propagação

# Propagação forward
def forward_propagation(X, parameters):
    
    # Lista de valores anteriores (cache)
    caches = []
    
    # Dados de entrada
    A = X
    
    # Comprimento dos parâmetros
    L = len(parameters) // 2
   
    # Loop
    for i in range(1, L):
      
        # Guarda o valor prévio de A
        A_prev = A
        
        # Executa o forward
        A, cache = forward(A_prev, parameters["W" + str(i)], parameters["b" + str(i)], activation = "relu")
        
        # Grava o cache
        caches.append(cache)
    
    # Saída na última camada
    A_last, cache = forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation = "sigmoid")
    
    # Grava o cache
    caches.append(cache)
    
    return(A_last, caches)

### Função de Custo (Cost Function)

In [8]:
def calc_custo(A_last, Y):
    
    # Ajusta o shape de Y para obter seu comprimento (total de elementos)
    m = Y.shape[1]
    
    # Calcula o custo comparando valor real e previso
    custo = (-1 / m) * np.sum((Y * np.log(A_last)) + ((1 - Y) * np.log(1 - A_last)))
    
    # Ajusta o shape do custo
    custo = np.squeeze(custo)
    
    return(custo)

### Backward Propagation - Função Sigmóide Backward

In [9]:
# Função sigmoid para o backpropagation 
# Cálculo da derivada pois temos interessa na informação relacionada a sua variação
def sigmoid_backward(da, Z):
    
    # Calculo da derivada de Z
    dg = (1 / (1 + np.exp(-Z))) * (1 - (1 / (1 + np.exp(-Z))))
    
    # Identificação da alteração na derivada de z
    dz = da * dg
    return dz

#### Backward Propagation - Função ReLu Backward

In [10]:
def relu_backward(da, Z):
    
    dg = 1 * ( Z >= 0)
    dz = da * dg
    return dz

#Os interesses são os mesmos da função sigmóide no que se refere a informação d avriação.

### Backward Propagation - Ativação Linear Backward

In [11]:
def linear_backward_function(dz, cache):
    
    # Recebe valor da memória (cache)
    A_prev, W, b = cache
    
    # Shape de m
    m = A_prev.shape[1]
    
    # Calculo da derivada de W (Result. operação com dz)
    dW = (1 / m) * np.dot(dz, A_prev.T)
    
    # Calculo da derivada de b (Result. operação com dz)
    db = (1 / m) * np.sum(dz, axis = 1, keepdims = True)
    
    # Calculo da derivada da operação
    dA_prev = np.dot(W.T, dz)
    
    return dA_prev, dW, db

### Backward Propagation - Ativação Linear Backward

In [12]:
# Tipo de ativação (relu ou sigmoid)
def linear_activation_backward(dA, cache, activation):
    
    # Extrai o cache
    linear_cache, activation_cache = cache
    
    # Verifica se é ReLu
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward_function(dZ, linear_cache)
        
    # Verifica se é Sigmoid
    if activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward_function(dZ, linear_cache)
        
    return dA_prev, dW, db

### Algoritmo Backpropagation

In [13]:
def backward_propagation(AL, Y, caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    
    # Ajusta o shape de Y
    Y = Y.reshape(AL.shape)
    
    # Calcula a derivada da previsão final da rede(ForwardPropagation) e em seguida recupera o valor atual do cache
    dAL = -((Y / AL) - ((1 - Y) / (1 - AL)))
    current_cache = caches[L - 1]
    
    # Geração da a lista de gradiente para: dados, pesos e bias
    grads["dA" + str(L - 1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    # Loop para calculao durante as ativações utilizando a ReLu
    for l in reversed(range(L - 1)):
        
        # Memória atual + Calc de derivadas.
        current_cache = caches[l]
        dA_prev, dW, db = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        
        # Dá infos para os gradientes, usando o índice relacionado
        grads["dA" + str(l)] = dA_prev
        grads["dW" + str(l + 1)] = dW
        grads["db" + str(l + 1)] = db
        
    return grads

### Gradientes e Pesos

In [14]:
# Atualização de pesos
def atualiza_pesos(parameters, grads, learning_rate):
    
    # Comprimento da estrutura de dados com os parâmetros (pesos e bias)
    L = len(parameters)//2
    
    # Loop para atualização dos pesos
    for l in range(L):
        
        # Atualização dos pesos
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - (learning_rate * grads["dW" + str(l + 1)])
        
        # Atualização do bias
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - (learning_rate * grads["db" + str(l + 1)])
    
    return parameters

#### Implementação da Rede

In [15]:
# Modelo completo da rede neural
def modeloNN(X, Y, dims_camada_entrada, learning_rate = 0.0075, num_iterations = 100):
    #Vejam o tamanho do learning_rate. Comentei durante em aula que as taxas são realmente baixas.
    
    # Lista para receber o custo a cada epoch em train
    custos = []
    
    # Inicializa parâmetros
    parametros = inicializa_parametros(dims_camada_entrada)
    
    # Loop pelo número de iterações (epochs)
    for i in range(num_iterations):
        
        # Forward Propagation
        AL, caches = forward_propagation(X, parametros)
        
        # Calculo do custo
        custo = calc_custo(AL, Y)
        
        # Backward Propagation
        gradientes = backward_propagation(AL, Y, caches)
        
        # Atualização depesos
        parametros = atualiza_pesos(parametros, gradientes, learning_rate)
        
        # Print do valor intermediário do custo, sendo que a redução do custo indica o aprendizado do modelo
        if i % 10 == 0:
            print("Custo Após " + str(i) + " iterações é " + str(custo))
            custos.append(custo)
            
    return parametros, custos 

In [17]:
# Função para fazer as previsões

def predict(X, parametros):
    AL, caches = forward_propagation(X, parametros)
    return AL

# Notem que não há backpropagation, pois conforme as previsões surgem, os valores dos pesos mais aderentes acabam ficando conhecidos.

In [18]:
cancer = load_breast_cancer()

print(type(cancer))

<class 'sklearn.utils.Bunch'>


In [20]:
cancerDF = pd.DataFrame(columns=load_breast_cancer()["feature_names"], data=load_breast_cancer()["data"])

In [21]:
cancerDF.shape

(569, 30)

In [22]:
cancerDF.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [23]:
target= load_breast_cancer()["target"]

In [24]:
type(target)

numpy.ndarray

In [25]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [26]:
print("Total da Classe do tipo Benigno", np.count_nonzero(target==1))
print("="*15)
print("Total da Classe do tipo Maligno", np.count_nonzero(target==0))

Total da Classe do tipo Benigno 357
Total da Classe do tipo Maligno 212


In [32]:
target_names = load_breast_cancer()["target_names"]
type(target_names)

numpy.ndarray

In [33]:
labels = {}

target_names = load_breast_cancer()["target_names"]

#Mapeamento
for i in range(len(target_names)):
    labels.update({i:target_names})

In [34]:
#Labels - Lembrete: Sempre trabalhamos com número
labels

{0: array(['malignant', 'benign'], dtype='<U9'),
 1: array(['malignant', 'benign'], dtype='<U9')}

In [35]:
X = np.array(cancerDF)

In [40]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, target, test_size=0.20, shuffle=True)

In [48]:
y_treino

array([0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,

In [49]:
# Verificação de estágio dos nossos dados X, y(Treino)
print("Dados Treino")
print(X_treino.shape())
print(y_treino.shape())
print("="*20)
# Verificação de estágio dos nossos dados X, y(Teste)
print("Dados Teste")
print(X_teste.shape())
print(y_teste.shape())

Dados Treino


TypeError: 'tuple' object is not callable

In [50]:
# Ajuste nos dados de entrada
X_treino = X_treino.T
X_teste = X_teste.T

# Ajuste nos dados de saida
y_treino = y_treino.reshape(1, len(y_treino))
y_teste = y_teste.reshape(1, len(y_teste))

In [52]:
# Dimensões de entrada com numero de neuronios
dims_camada_entrada = [X_treino.shape[0], 50, 20, 5, 1]

In [53]:
# Treinando modelo

iniProc = time.time()
parametros, custo = modeloNN(X = X_treino,
                             y = y_treino,
                             dims_camada_entrada = dims_camada_entrada, num_iterations=3000, learning_rate=0.0075)

print("Treinamento durou %2.f segundos " % (time.time() - iniProc))

TypeError: modeloNN() got an unexpected keyword argument 'y'

In [ ]:
import seaborn as sns
# Plot do erro no treinamento
sns.set(rc={"figure.figuresize":(25,15)});
plt.plot(custo)
plt.title("Erros em Treinamento")

In [ ]:
y_pred_treino = predict(X_treino, parametros)

In [ ]:
y_pred_treino

In [ ]:
# Ajustamos o shape para o treino
y_pred_treino = y_pred_treino.reshape(-1)
y_treino = y_treino.reshape(-1)
y_pred_treino > 0.5

In [ ]:
# Conversão das previsões binário de classe (0 ou 1), usando o treshould o valor de 0.5 da probabilidade
y_pred_treino = 1 * (y_pred_treino > 0.5)
y_pred_treino

In [ ]:
aval_treino = sum(1 * (y_pred_treino == y_treino)) / len(y_pred_treino) * 100
print("Acurracia em Treino: " + str(y_pred_treino))

In [ ]:
print(classification_report(y_treino, y_pred_treino, target_names= ['Maligno', 'Benigno']))

In [ ]:
# Previsões usando dados de teste
y_pred_teste = predict(X_teste, parametros)
y_pred_teste

In [ ]:
# Ajustamos dos shapes
y_pred_teste = y_pred_teste.reshape(-1)
y_teste = y_teste.reshape(-1)

y_pred_teste = 1 * (y_pred_teste > 0.5)

y_pred_teste

In [ ]:
acuraciaFim = sum(1 * (y_pred_teste == y_teste)) / len(y_pred_teste) * 100
print("Acuracia em dados de Teste: " + str(acuraciaFim))

In [ ]:
print()